폴리백 수량계산
===

## 수량 계산과정

1. 오더번호에서 복종글자를 삭제한 항목을 만들고 리스트를 생성한다. (편의상 list_A 라고 호칭)
1. list_A 에서 항목을 하나씩 꺼낸다. (꺼낸 항목은 편의상 A라고 호칭)
    1. A는 3가지 경우로 나뉨 (오더번호가 3개, 2개 1개)
        1. 오더 3개의 경우
            1. H 복종의 지시량 * 1.02(LOSS)
            2. 남은 2복종의 1호 ~ 15호 세부수량을 비교하여 최대값을 누적
            3. 합계
        1. 오더 2개의 경우
            1. 두 오더의 1호 ~ 15호 세부수량을 비교하여 최대값을 누적
        1. 오더 1개의 경우
            1. 지시량 * 1.02(LOSS)

## ERP에서 자료 가져오기

In [1]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import binascii   # 한글 변환에 필요한 라이브러리
import sys
import math
import pyautogui
from datetime import datetime

In [2]:
sys.path.append('/settings')
import config

def connect_db(sid: str) -> 'sqlalchemy.engine.base.Engine':
    if sid != config.COMPANY_DB_CONFIG['sid']:
        raise ValueError("DB 를 찾을 수 없습니다.")
    
    conn = create_engine(
        "oracle+cx_oracle://{user}:{password}@{host}:{port}/{sid}?encoding=UTF-8&nencoding=UTF-8".format(
            user=config.COMPANY_DB_CONFIG['user'],
            password=config.COMPANY_DB_CONFIG['password'],
            host=config.COMPANY_DB_CONFIG['host'],
            port=config.COMPANY_DB_CONFIG['port'],
            sid=config.COMPANY_DB_CONFIG['sid']
        )
    )
    return conn

engine = connect_db('IVY')

## 사용자 입력값

In [3]:
confirm: str = pyautogui.confirm(title='알림', text='작업을 시작합니다.\
\n취소하면 작업을 중단합니다.\
\n\n< 기본적용 옵션 >\
\n1. 체육복 오더만 조회\
\n\n작업이 완료되면 엑셀파일을 확인하세요.')

if confirm == 'Cancel':
    sys.exit()

In [4]:
today: str = datetime.today().strftime("%Y%m%d")

In [5]:
start_date: str = pyautogui.prompt(title='조회시작일자', text=f'조회시작일자를 입력하세요. 예 : {today}', default=today)

In [6]:
# 오입력 검증
while True:
    i: int = 0
    if len(start_date) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        start_date = pyautogui.prompt(title='조회시작일자', text=f'조회시작일자를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

In [7]:
end_date: str = pyautogui.prompt(title='조회종료일자', text=f'조회종료일자를 입력하세요. 예 : {today}', default=today)

In [8]:
# 오입력 검증
while True:
    i: int = 0
    if len(start_date) != 8:
        i += 1
        pyautogui.alert(title='입력오류', text='날짜는 8자리 입니다.')
        end_date = pyautogui.prompt(title='조회종료일자', text=f'조회종료일자를 입력하세요. 예 : {today}', default=today)

    if i == 0:
        break 

In [9]:
start_date = start_date[:4] + '/' + start_date[4:6] + '/' + start_date[-2:]
end_date = end_date[:4] + '/' + end_date[4:6] + '/' + end_date[-2:]

start_date, end_date

('2022/01/01', '2022/07/26')

### 한글변환 함수
    - SQL문의 한글 컬럼에는 다음과 같이 함수로 처리
    - rawtohex(utl_raw.cast_to_raw(변환할 컬럼))

In [10]:
# US7ASCII -> CP949(완성형한글) 로 변환
def us7ascii_to_cp949(df: pd.DataFrame) -> pd.DataFrame:
    for index, byte_data in enumerate(df):
        if byte_data == None: # null 값이면 패스. 안하면 변환 에러난다.
            continue
        byte_data = binascii.unhexlify(df[index])  # 16진수 문자열 hexstr로 표현된 바이너리 데이터를 반환. 역함수는 b2a_hex()
        df[index] = byte_data.decode("cp949")  # 바이트 변환값 -> cp949(완성형 한글) 로 변환
    return df

## 타입오더 데이터 가져오기

In [11]:
def count_polybag() -> pd.DataFrame:
    oracleSql: str = f'''
SELECT rawtohex(utl_raw.cast_to_raw(tkyk_name)) tkyk_name,
       rawtohex(utl_raw.cast_to_raw(agen_name)) agen_name,
       rawtohex(utl_raw.cast_to_raw(sch_name))sch_name,
       master_order,
       master_bokjong,
       Of_sojae_st_set(master_status,master_grade_cnt, b.bkjk_sojae_hgb, b.bkjk_sojae, b.bkjk_sojae_2h, b.bkjk_sojae_3h, s1.stand_sojae) sojae,
       To_char(Nvl(detail_qty1,0)) no_1,
       To_char(Nvl(detail_qty2,0)) no_2,
       To_char(Nvl(detail_qty3,0)) no_3,
       To_char(Nvl(detail_qty4,0)) no_4,
       To_char(Nvl(detail_qty5,0)) no_5,
       To_char(Nvl(detail_qty6,0)) no_6,
       To_char(Nvl(detail_qty7,0)) no_7,
       To_char(Nvl(detail_qty8,0)) no_8,
       To_char(Nvl(detail_qty9,0)) no_9,
       To_char(Nvl(detail_qty10,0)) no_10,
       To_char(Nvl(detail_qty11,0)) no_11,
       To_char(Nvl(detail_qty12,0)) no_12,
       To_char(Nvl(detail_qty13,0)) no_13,
       To_char(Nvl(detail_qty14,0)) no_14,
       To_char(Nvl(detail_qty15,0)) no_15,
       To_char(master_jisi_qty) jisi_qty,
       To_char(Nvl(jts_f_qty1,0) + Nvl(jts_f_qty2,0) + Nvl(jts_f_qty3,0) + Nvl(jts_f_qty4,0) + Nvl(jts_f_qty5,0) + Nvl(jts_t_qty1,0) + Nvl(jts_t_qty2,0) + Nvl(jts_t_qty3,0) + Nvl(jts_t_qty4,0) + Nvl(jts_t_qty5,0)) t_qty,
       To_char(fact_date,'yy/mm/dd') fact_date
FROM   i_suju_master_t,
       i_qty_detail_t,
       i_tkyk_t,
       i_agen_t,
       i_sch_t,
       i_suju_stand_t s1,
       i_stand_bkjk_t b,
       i_suju_fact_t
WHERE  master_status >= '50'
AND    master_tkyk = tkyk_code(+)
AND    master_agent = agen_code(+)
AND    master_school= sch_code(+)
AND    detail_gbn = '2'
AND    detail_order = master_order
AND    fact_order = master_order
AND    master_jaepum = 'F'
AND    b.bkjk_squota(+) = master_squota
AND    b.bkjk_school(+) = master_school
AND    b.bkjk_bokjong(+) = master_bokjong
AND    s1.stand_order(+) = master_order
AND    s1.stand_sojae_gbn(+)= '1'
AND    fact_date >= To_date('{start_date}','yyyy/mm/dd')
AND    fact_date <= To_date('{end_date}','yyyy/mm/dd')
ORDER BY master_order
'''
# AND    substr(fact_order,1,3) IN ('22F')
    df: pd.DataFrame = pd.read_sql_query(oracleSql, engine)
    df_temp: pd.DataFrame = df['tkyk_name'].copy()
    df['tkyk_name'] = us7ascii_to_cp949(df_temp)
    
    df_temp: pd.DataFrame = df['agen_name'].copy()
    df['agen_name'] = us7ascii_to_cp949(df_temp)
    
    df_temp: pd.DataFrame = df['sch_name'].copy()
    df['sch_name'] = us7ascii_to_cp949(df_temp)
    
    return df

In [12]:
df = count_polybag()

In [13]:
df.columns = ['특약', '대리점명', '학교명', '오더', '복종', '소재',
              '1호', '2호', '3호', '4호', '5호', '6호', '7호', '8호', '9호', '10호', '11호', '12호', '13호', '14호', '15호',
              '지시량', '특이수량', '타입일']

In [14]:
df.iloc[:, 6:23] = df.iloc[:, 6:23].astype(int) # 데이터 타입 일괄 변경
df.dtypes

특약      object
대리점명    object
학교명     object
오더      object
복종      object
소재      object
1호       int32
2호       int32
3호       int32
4호       int32
5호       int32
6호       int32
7호       int32
8호       int32
9호       int32
10호      int32
11호      int32
12호      int32
13호      int32
14호      int32
15호      int32
지시량      int32
특이수량     int32
타입일     object
dtype: object

In [15]:
type_list = list(set(df['타입일'].tolist())) # 중복제거
type_list.sort() # 일자별 정렬
len(type_list)

49

### 폴리백 수량 계산

In [16]:
poly_cnt: int = 0 # 반복문 돌며 각 세부수량 누적
df_total = pd.DataFrame() # 최종 데이터프레임

for type_dt in type_list: # 타입일자별로 쪼개기
    df_type = df[df['타입일'] == type_dt].copy()
    df_type['오더(비교용)'] = df_type['오더'].str[:8] + df_type['오더'].str[10:] # 비교용 컬럼
    
    # H 복종
    df_type_H = df_type[df_type['복종'] == 'H'].copy()
    df_type_H['복종수'] = 1
    df_type_H['폴리백'] = np.ceil(df_type_H['지시량'] * 1.02) # pandas에 ceil 함수가 없음. round 만 존재.
    df_total = pd.concat([df_total, df_type_H])
    
    # F, W 복종
    df_type_FW = df_type[df_type['복종'] != 'H'].copy()
    
    poly_list = list(set(df_type_FW['오더(비교용)'].tolist())) # 중복제거
    
    for p_list in poly_list:
        df_temp = df_type_FW[df_type_FW['오더(비교용)'] == p_list].copy() # 그룹단위 반복문
        if len(df_temp) == 2:
            for i in range(6, 21):
                poly_cnt = poly_cnt + df_temp.iloc[:, i].max()
            poly_cnt = math.ceil(poly_cnt * 1.02)
        elif len(df_temp) == 1:
            poly_cnt = math.ceil(df_temp['지시량'] * 1.02)
        else:
            print(f'복종수가 {len(df_temp)}개는 처리할 수 없습니다.')
            sys.exit()

        df_temp['복종수'] = len(df_temp)
        df_temp['폴리백'] = poly_cnt
        
        df_total = pd.concat([df_total, df_temp])

        poly_cnt = 0 # 변수 초기화
        poly_cnt_H = 0

In [17]:
df_total = df_total.sort_values(['타입일', '오더(비교용)', '복종'], ascending=[True, True, False])
df_total

,특약,대리점명,학교명,오더,복종,소재,1호,2호,3호,4호,...,12호,13호,14호,15호,지시량,특이수량,타입일,오더(비교용),복종수,폴리백
355,중부상권,일산점,백석중(일산),22NAA090H 2,H,HCM0991,0,2,6,8,...,0,0,0,0,32,0,22/01/05,22NAA0902,1,33.0
555,중부상권,안산점,신길중,22NAA617W 1,W,HCR0204,0,0,2,6,...,0,0,0,0,40,0,22/01/05,22NAA6171,2,41.0
554,중부상권,안산점,신길중,22NAA617F 1,F,HCR0204,0,0,2,6,...,0,0,0,0,40,0,22/01/05,22NAA6171,2,41.0
652,중부상권,송탄점,태광고,22NAC137W 2,W,HCR0580,0,11,7,0,...,0,0,0,0,75,0,22/01/05,22NAC1372,2,77.0
651,중부상권,송탄점,태광고,22NAC137F 2,F,HCR0580,0,11,7,0,...,0,0,0,0,75,0,22/01/05,22NAC1372,2,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,서울상권,서대문점,충암초등(기능성),22FSE053W 1,W,HCR0306,0,80,60,60,...,0,0,0,0,260,0,22/07/23,22FSE0531,2,368.0
273,서울상권,서대문점,충암초등(기능성),22FSE053F 1,F,HCR0306,0,90,80,80,...,0,0,0,0,360,0,22/07/23,22FSE0531,2,368.0
279,서울상권,서대문점,충암유치원,22FSG001W 1,W,HCR0306,0,6,30,30,...,0,0,0,0,100,0,22/07/23,22FSG0011,2,102.0
278,서울상권,서대문점,충암유치원,22FSG001F 1,F,HCR0306,0,6,30,30,...,0,0,0,0,100,0,22/07/23,22FSG0011,2,102.0


In [18]:
df_total = df_total.groupby(['타입일',
                             '오더(비교용)',
                             '특약',
                             '대리점명',
                             '학교명',
                             '폴리백',
                             '복종수',
                             '오더',
                             '복종',
                             '소재',
                             '1호',
                             '2호',
                             '3호',
                             '4호',
                             '5호',
                             '6호',
                             '7호',
                             '8호',
                             '9호',
                             '10호',
                             '11호',
                             '12호',
                             '13호',
                             '14호',
                             '15호',
                             '특이수량',
                             '지시량']).count()

In [19]:
df_total = df_total.reset_index('오더(비교용)', drop=True)

In [20]:
file_name = f'{start_date.replace("/", "")}_{end_date.replace("/", "")}_폴리백수량.xlsx'

In [21]:
with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    df_total.to_excel(writer, sheet_name='폴리백수량')

## xlwings 로 파일 수정

In [22]:
import xlwings as xw

In [23]:
app = xw.App(visible=False) # 실행과정 안보이게

In [24]:
wb = xw.Book(file_name) # type: xlwings.main.Book
sh = wb.sheets['폴리백수량'] # type: xlwings.main.Sheet

In [25]:
sh.range('1:1').delete() # 첫 행 삭제

### 행 최대값, 열 최대값 구하기

- expand 나 option을 폴리백 컬럼에 직접 쓰면 null 값에서 멈춘다. (3칸 나옴)
- 모든 값이 기록된 곳에서 최대값을 구한다.

In [26]:
max_row = len(sh.range('G1').expand('down').value)
max_col = len(sh.range('A2').expand('right').value)
max_row, max_col

(2251, 26)

In [27]:
sh.range((1, 1), (max_row, max_col)).font.bold = False # bold 없애기

### COPY and PASTE

In [28]:
sh.range(f'E1:E{max_row}').copy()
sh.range((1, max_col+1)).paste(paste='all')

### AUTOFIT

In [29]:
for ws in wb.sheets:
    ws.autofit(axis="columns")

In [30]:
wb.save()

In [31]:
wb.close()

In [32]:
app.kill()

In [33]:
pyautogui.alert(title='알림', text='작업이 종료되었습니다.\n엑셀파일을 확인하세요.')

'OK'